# Manual Multi-Point measurement

This notebook is designed to repeat a measurement at multiple user-defined positions.

The list of positions can be set by hand by moving the stage and repeatedly getting the current stage coordinates from Imspector.

In [ ]:
import specpy

from pipeline2.taskgeneration.coordinate_building_blocks import StagePositionListGenerator, StageOffsetsSettingsGenerator
from pipeline2.callback_buildingblocks.static_settings import JSONSettingsLoader
from pipeline2.taskgeneration import AcquisitionTaskGenerator
from pipeline2.pipeline import AcquisitionPipeline
from pipeline2.imspector import get_current_stage_coords

## 1: Parameters

For this automation pipeline, we only need to know where to save our data and which settings to use for each position

In [ ]:
# where to save & whether to save combined HDF5 file
save_folder = 'D:/AUTOMATION/TEST/multipoint'
save_hdf5 = True

# path of measurement parameters (dumped to JSON file)
measurement_parameters = 'C:/Users/RESOLFT/Desktop/config_json/20200921_2colorFISH_ov.json'

### 1a (OPTIONAL): Use settings from current measurement in Imspector

Run this cell to re-use the settings of current measurement (and configuration) in Imspector instead of loading them from a file.

In [ ]:
imspector = specpy.get_application()
measurement_parameters = imspector.active_measurement().parameters('')

### 1b (OPTIONAL): Use settings from current measurement in Imspector (multiple configurations)

Run this cell to re-use the settings of current measurement in Imspector.

In contrast to 1a, this will use all configurations of the current measurement and run them all at each position in the multipoint experiment. 

In [ ]:
measurement_parameters = []

measurement = imspector.active_measurement()
for configuration in measurement.configuration_names():
    measurement.activate(measurement.configuration(configuration))
    measurement_parameters.append(measurement.parameters(''))

## 2: Building list of coordinates to image

Now, we manually build a list of stage coordinates to image.

Run the second cell multiple times (after moving to positions you want to image at) to add multiple points.

In [ ]:
# start with empty position list
coordinate_list = []

In [ ]:
# append current stage position, run multiple times to enqueue multiple positions to image at
coordinate_list.append(get_current_stage_coords())

In [ ]:
# optionally reverse position list (saves initial move if we start right after setting last position)
coordinate_list = coordinate_list[::-1]

In [ ]:
# show for verification
coordinate_list

## 3: run multipoint acquisition

In [ ]:
# callback that will create an acquisition task with given measurement parameters
# at the next stage coordinates in the coordinate list (the next 'position')
next_position_generator = AcquisitionTaskGenerator('position',
    JSONSettingsLoader(measurement_parameters, as_measurements=False),
    StageOffsetsSettingsGenerator(StagePositionListGenerator(coordinate_list)))

# build pipeline object (1 level: 'position')
pipeline = AcquisitionPipeline('multipoint-acquisition', save_folder, ['position'], save_combined_hdf5=save_hdf5)
# attach callback so that after each position, the next one will be enqueued
pipeline.add_callback(next_position_generator, 'position')

# start with initial task from callback
pipeline.run(next_position_generator)